<a href="https://colab.research.google.com/github/sixvo-labs/Sapir/blob/main/Copy_of_OpenAI_Multilingual_ASR_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Whisper

The commands below will install the Python packages needed to use Whisper models and evaluate the transcription results.

In [ ]:
! pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-f96j_5pd
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-f96j_5pd
     |████████████████████████████████| 4.9 MB 4.7 MB/s 
     |████████████████████████████████| 120 kB 54.7 MB/s 
     |████████████████████████████████| 6.6 MB 55.7 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1174001 sha256=c91ad658bfcfcf0973236181ff7579b1a481f544d33623f1940afa2124ce7b1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-x1v6qco5/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper


In [ ]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm


pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

# Record and transcribe

In [ ]:
# from https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=10):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  return 'audio.wav'  # or webm ?

In [ ]:
record()

<IPython.core.display.Javascript object>

'audio.wav'

In [ ]:
from IPython.display import Audio
from IPython.display import display
wn = Audio('audio.wav', autoplay=True) ##
display(wn)##


In [ ]:

model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("audio.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 103MiB/s]


Model is multilingual and has 762,321,920 parameters.
Detected language: iw
ועל פיסת החרס כתוב כך, לפעמים נדמה כאילו נכתבו כבר כל השירים כולם.
